# Python to SQL, and back again
In this codealong we will show you how to create a relational database from your pandas DataFrames.
> **To run this notebook you will need to work locally and not on colab.**

---
## 1.&nbsp; Import libraries 💾
If you haven't already installed sqlalchemy, you will need to. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [1]:
# install if needed
# !pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.1 MB 4.3 MB/s eta 0:00:01
   -------- ------------------------------- 0.4/2.1 MB 5.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.1 MB 6.0 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.1 MB 4.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 6.3 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/292.3 kB ? eta -:--:--
   ---------------------------------------  286.7/292.3 kB 8.9 MB/s eta 0:00:01
   ---------------------------------------- 292.3/292.3 kB 6.0 MB/s eta 0:00:00


In [2]:
import pandas as pd

---
## 2.&nbsp; Relational Databases 📂

Creating DataFrames in python and pandas often results in tables with repeated information, as shown in the example below.
<br>

| author_name | book_title | year_published |
| --- | --- | --- |
| Arthur Conan Doyle | The Adventures of Sherlock Holmes | 1887 |
| J.R.R. Tolkien | The Hobbit | 1937 |
| J.R.R. Tolkien | The Lord of the Rings | 1954 |
| Harper Lee | To Kill a Mockingbird | 1960 |
| Harper Lee | Go Set a Watchman | 2015 |
<br>

This can be problematic for relational databases, which are designed to store data efficiently and avoid redundancy. To address this issue, we will separate the author and book information into two tables: authors and books. This approach eliminates duplicate data, ensuring data integrity and optimising storage.
<br>

| author_id | author_name |
| --- | --- |
| 1 | Arthur Conan Doyle |
| 2 | J.R.R. Tolkien |
| 3 | Harper Lee |
<br>

| book_id | book_title | year_published | author_id |
|---|---|---|---|
| 1 | The Adventures of Sherlock Holmes | 1887 | 1 |
| 2 | The Hobbit | 1937 | 2 |
| 3 | The Lord of the Rings | 1954 | 2 |
| 4 | To Kill a Mockingbird | 1960 | 3 |
| 5 | Go Set a Watchman | 2015 | 3 |

---
## 3.&nbsp; Creating the authors table with python 🐍
Let's start by creating the original DataFrame, including the repeated data.

In [6]:
names = ["Arthur Conan Doyle", "J.R.R. Tolkien", "J.R.R. Tolkien", "Harper Lee", "Harper Lee"]
titles = ["The Adventures of Sherlock Holmes", "The Hobbit", "The Lord of the Rings", "To Kill a Mockingbird", "Go Set a Watchman"]
years = [1887, 1937, 1954, 1960, 2015]

non_relational_df = pd.DataFrame({"author_name": names,
                                  "book_title": titles,
                                  "year_published": years})

non_relational_df

,author_name,book_title,year_published
0,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887
1,J.R.R. Tolkien,The Hobbit,1937
2,J.R.R. Tolkien,The Lord of the Rings,1954
3,Harper Lee,To Kill a Mockingbird,1960
4,Harper Lee,Go Set a Watchman,2015


Now, let's select only the authors without any duplicates.

In [7]:
authors_unique = non_relational_df["author_name"].unique()

authors_df = pd.DataFrame({"author_name": authors_unique})

authors_df

,author_name
0,Arthur Conan Doyle
1,J.R.R. Tolkien
2,Harper Lee


Fantastic! This DataFrame will be the foundation of our authors table.

---
## 4.&nbsp; Creating the matching authors table with SQL 💻

Ok, now we're ready to store this DataFrame in SQL. Before we can send the information in SQL, we need to make a table that has the same columns and data types to recieve the data. While we are creating a table for authors, we can also create the books table too.

Open MySQL Workbench, open a local connection, and open a new file. Then copy and paste the code from below.

```sql
-- Drop the database if it already exists
DROP DATABASE IF EXISTS sql_workshop ;

-- Create the database
CREATE DATABASE sql_workshop;

-- Use the database
USE sql_workshop;

-- Create the 'authors' table
CREATE TABLE authors (
    author_id INT AUTO_INCREMENT, -- Automatically generated ID for each author
    author_name VARCHAR(255) NOT NULL, -- Name of the author
    PRIMARY KEY (author_id) -- Primary key to uniquely identify each author
);

-- Create the 'books' table
CREATE TABLE books (
    book_id INT AUTO_INCREMENT, -- Automatically generated ID for each book
    book_title VARCHAR(255) NOT NULL, -- Title of the book
    year_published INT, -- Year the book was published
    author_id INT, -- ID of the author who wrote the book
    PRIMARY KEY (book_id), -- Primary key to uniquely identify each book
    FOREIGN KEY (author_id) REFERENCES authors(author_id) -- Foreign key to connect each book to its author
);
```

To download the sql file that we will follow for this section, [click here](https://drive.google.com/uc?export=download&id=1tln_33FM7D9wLckzxacBJNcMYqtyxybE)

If you'd like more information about MySQL data types [click here](https://www.w3schools.com/mysql/mysql_datatypes.asp).

---
## 5.&nbsp; Sending the information from this notebook to sql 📠
To establish a connection with the SQL database, we need to provide the notebook with the necessary information, which we do using the connection string below. You will need to modify only the password variable, which should match the password you set during MySQL Workbench installation.

In [3]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = "MySQLjau"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

To send information to our sql databse we use the pandas method `.to_sql()`. The argument `if_exists="append"` says that we don't want to overwrite any existing data, but add on to what is already there.

In [11]:
authors_df.to_sql('authors',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

Now, have a look at the table `authors` in MySQL Workbench, you should see that the names of the authors have appeared.

---
## 6.&nbsp; Retrieving information from sql to this notebook 📥
It's not only possible to send information to a SQL database, but also retrieve it too. Using `.read_sql()` in combination with the `connection_string` we can access the required data.

In [13]:
authors_from_sql = pd.read_sql("authors", con=connection_string)
authors_from_sql

,author_id,author_name
0,1,Haruki Murakami
1,2,Margaret Atwood
2,3,Henrik Ibsen
3,4,Arthur Conan Doyle
4,5,J.R.R. Tolkien
5,6,Harper Lee


Using this same method, we can also perform SQL queries to only bring back certain sections of information instead of the whole DataFrame.

In [14]:
pd.read_sql("""
            SELECT DISTINCT author_name
            FROM authors
            """,
            con=connection_string)

,author_name
0,Haruki Murakami
1,Margaret Atwood
2,Henrik Ibsen
3,Arthur Conan Doyle
4,J.R.R. Tolkien
5,Harper Lee


---
## 7.&nbsp; Preparing and sending the books table 📚
By extracting the authors table from our SQL database, we gain access to the unique identifier `author_id` assigned to each author. These `author_id`'s serve as pointers to their corresponding author records, allowing us to seamlessly link the `author_id`'s in the books table to their respective authors in the authors table, thereby completing the books table.

In [20]:
books_df = non_relational_df.merge(authors_from_sql,
                                   on = "author_name",
                                   how="left")

books_df

,author_name,book_title,year_published,author_id
0,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887,4
1,J.R.R. Tolkien,The Hobbit,1937,5
2,J.R.R. Tolkien,The Lord of the Rings,1954,5
3,Harper Lee,To Kill a Mockingbird,1960,6
4,Harper Lee,Go Set a Watchman,2015,6


In [21]:
books_df = books_df.drop(columns=["author_name"])

books_df

,book_title,year_published,author_id
0,The Adventures of Sherlock Holmes,1887,4
1,The Hobbit,1937,5
2,The Lord of the Rings,1954,5
3,To Kill a Mockingbird,1960,6
4,Go Set a Watchman,2015,6


In [22]:
books_df.to_sql('books',
                if_exists='append',
                con=connection_string,
                index=False)

5

In [23]:
books_from_sql = pd.read_sql("books", con=connection_string)
books_from_sql

,book_title,year_published,author_id
0,The Adventures of Sherlock Holmes,1887,4
1,The Hobbit,1937,5
2,The Lord of the Rings,1954,5
3,To Kill a Mockingbird,1960,6
4,Go Set a Watchman,2015,6
5,The Adventures of Sherlock Holmes,1887,4
6,The Hobbit,1937,5
7,The Lord of the Rings,1954,5
8,To Kill a Mockingbird,1960,6
9,Go Set a Watchman,2015,6


---
## 8.&nbsp; Challenge 😃
Now that you've learnt how to send and retrieve information, it's your turn to show off your skills. Create multiple tables in SQL for the data you scrapped about cities from Wikipedia. One should just be a table about the cities, the others should be facts about the cities.

| city_id | city |
| --- | --- |
| 1 | Berlin |
| 2 | Hamburg |
| 3 | Munich |

<br>

| City ID | Population | Year Data Retrieved |
|---|---|---|
| 1 | 3,850,809 | 2024 |
| 2 | 1,945,532 | 2024 |
| 3 | 1,512,491 | 2024 |

> **Pro Tip:** Visualise your relational database with pen and paper before you start coding. This can help you to identify any potential problems or inconsistencies in your design, and it can also make the coding process more efficient.

In [68]:
from city_scrap import scrape_city_data

In [69]:
scrape_city_data = scrape_city_data()

In [72]:
scrape_city_data


,Country,City,Population,Lat,Lon,Elevation_in_m
0,Germany,Berlin,3576873,52°31′12″N,13°24′18″E,34.0
1,Germany,München,1512491,48°08′15″N,11°34′30″E,520.0
2,Germany,Hamburg,1945532,53°33′N,10°00′E,NaN
3,Germany,Bremen,563290,53°04′33″N,08°48′26″E,12.0


In [56]:
cities = scrape_city_data["City"].unique()
cities_df = pd.DataFrame({"City": cities})
cities_df

,City
0,Berlin
1,München
2,Hamburg
3,Bremen


In [74]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "MySQLjau"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [80]:
import sqlalchemy
import numpy
import sqlite3

In [86]:
cities_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

4

In [87]:
cities = pd.read_sql("cities", con=connection_string)
cities

,City_id,City
0,1,Berlin
1,2,München
2,3,Hamburg
3,4,Bremen


In [ ]:
cities_d.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [91]:
city_data = city_data.merge(cities,
                                   on = "City",
                                   how="inner")

city_data

,Country,City,Population,Lat,Lon,Elevation_in_m,City_id
0,Germany,Berlin,3576873,52°31′12″N,13°24′18″E,34.0,1
1,Germany,München,1512491,48°08′15″N,11°34′30″E,520.0,2
2,Germany,Hamburg,1945532,53°33′N,10°00′E,NaN,3
3,Germany,Bremen,563290,53°04′33″N,08°48′26″E,12.0,4


In [92]:
city_data = city_data.drop(columns=["City"])

In [94]:
city_data["Time_data_retrieved"] = ["05-03-2024", "05-03-2024", "05-03-2024", "05-03-2024"]
city_data

,Country,Population,Lat,Lon,Elevation_in_m,City_id,Time_data_retrieved
0,Germany,3576873,52°31′12″N,13°24′18″E,34.0,1,05-03-2024
1,Germany,1512491,48°08′15″N,11°34′30″E,520.0,2,05-03-2024
2,Germany,1945532,53°33′N,10°00′E,NaN,3,05-03-2024
3,Germany,563290,53°04′33″N,08°48′26″E,12.0,4,05-03-2024


In [95]:
city_data["Time_data_retrieved"] = pd.to_datetime(city_data['Time_data_retrieved'])

In [96]:
city_data.to_sql('city_data',
                  if_exists='append',
                  con=connection_string,
                  index=False)

4